In [ ]:
import rioxarray as rio
import os
import xarray as xr
from geocube.api.core import make_geocube
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import utils
import pickle
import rasterio
from tqdm.notebook import tqdm
import glob
import shapely.geometry as sgeom

from osgeo import gdal
gdal.SetConfigOption('GTIFF_SRS_SOURCE', 'EPSG')

Above imports are not all used!

Set the folders and paths

In [ ]:
ROOT = '/home/jovyan/work/project_data/landcover/model_outputs/final'
aoi = 'nlees_lcm_12pt5_rgb'

new_res = [1, 1]
new_res_str = f'{new_res[0]}x{new_res[1]}'
fname= f'{aoi}_reduce_{new_res_str}'
out_reduce_dir = os.path.join(
    ROOT,
    fname
)

out_paths = {
    'out_all_dir': os.path.join(ROOT, aoi),
}

New mapping for integer labels, here I am just using the numbers!

In [ ]:
labs = [
    0, 
    101,
    102,
    103,
    104,
    201,
    202,
    203,
    204,
    205,
    206,
    207,
    302,
    303,
    401,
    402,
    403,
    404,
    405,
    406,
    407,
    408,
    409,
    410,
    411,
    504,
]

new_map = dict(zip(labs, [str(l) for l in labs]))
reducer = utils.TileSquidger(files, [], {})
reducer.set_new_mapping(new_map)

The `reducer` object uses the list of files to iterate when processing.

In [ ]:
# List of tiles
paths = glob.glob(
    os.path.join(out_paths['out_all_dir'], '*.tif')
)
files = [os.path.split(p)[1] for p in paths]
#files

Run the reducer to create a folder with reduced tiles, must be multiples of input resolution

In [ ]:
if os.path.exists(out_reduce_dir):
    print(f'Overwriting {out_reduce_dir}')
else:
    os.makedirs(out_reduce_dir)

reducer.tile_reduce(out_paths['out_all_dir'], out_reduce_dir, new_res)

Create a `vrt` and reduce again if necessary to get a single file output

In [ ]:
out_vrt = out_reduce_dir+'.vrt'

In [ ]:
def create_vrt_from(files, out_fp, overviews=False, srs='EPSG:27700'):
    vrt_options = gdal.BuildVRTOptions(outputSRS=srs)
    ds = gdal.BuildVRT(out_fp, files,)# options=vrt_options)
    ds.FlushCache()
    del ds
    
    names = {}
    first = gdal.Open(files[0], 0)
    for i in range(1, first.RasterCount+1):
        names[i] = first.GetRasterBand(i).GetDescription()
    del first
    
    image = gdal.Open(out_fp, 1)
    for i in names.keys():
        b = image.GetRasterBand(i)
        b.SetDescription(names[i])
    del image
            
    if overviews:
        image = gdal.Open(out_fp, 0)
        gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
        image.BuildOverviews('NEAREST', [4, 8, 16, 32, 64, 128], gdal.TermProgress_nocb)
        del image

r_files = glob.glob(os.path.join(out_reduce_dir, '*.tif'))
create_vrt_from(r_files, out_vrt, overviews=False)

In [ ]:
img = rio.open_rasterio(
    out_vrt,
    masked=False,
    cache=False,
)

out_img = utils.reduce_to_proportion(img, [10, 10], boundary='trim', as_counts=True)

In [ ]:
out_path = os.path.join(ROOT, 'nlees_lcm_10m.tif')
out_img.astype('int32').rio.to_raster(out_path)